# BAX422 Final Project
### Group 10: Yi Yin (Ian) Chen/Priyanka Malhotra/Sushma Niveni Pindiga

In [1]:
# Import Packages
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd
import re

In [2]:
# Driver setup
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

In [3]:
# Maximize Window
driver.maximize_window() 
driver.minimize_window()  
driver.maximize_window()  
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [4]:
# Navigate to LinkedIn
driver.get('https://www.linkedin.com/login')
time.sleep(10)

In [5]:
with open('user_credentials.txt', 'r', encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]

user_name = user_credentials[0]  # First line is the username
password = user_credentials[1]  # Second line is the password

# Input the username and password and click on the login button
driver.find_element(By.ID, 'username').send_keys(user_name)
driver.find_element(By.ID, 'password').send_keys(password)
time.sleep(1)

login_button = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')
login_button.click()
driver.implicitly_wait(30)

In [6]:
# Jobs page
jobs_button = driver.find_element(By.XPATH, "//span[@title='Jobs']")
jobs_button.click()
time.sleep(3)

In [7]:
# Directly go to search results
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3866029496&f_E=2%2C4&f_JT=F&f_SB2=1&geoId=103644278&keywords=Data%20Analyst&location=United%20States")
time.sleep(1)

In [10]:
# Get all links for these offers
base_url = "https://www.linkedin.com/jobs/search/?currentJobId=3866029496&f_E=2%2C4&f_JT=F&f_SB2=1&geoId=103644278&keywords=Data%20Analyst&location=United%20States&start={offset}"

links = []
print('Links are being collected now.')

try:
    # Assuming each page lists 25 jobs and starting from the first page with an offset of 0
    for offset in range(0, 25 * 50, 25):  # Adjust 38 to the number of pages you want to scrape
        # Navigate directly to the desired page by setting the start parameter
        driver.get(base_url.format(offset=offset))
        time.sleep(5)  # Allow some time for the page to load

        jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                href = a.get_attribute('href')
                if href.startswith("https://www.linkedin.com/jobs/view") and href not in links:
                    links.append(href)
            driver.execute_script("arguments[0].scrollIntoView();", job)
        print(f'Collecting the links starting from job: {offset}')

except Exception as e:
    print(f"An exception occurred: {e}")
print(f'Found {len(links)} links for job offers.')


Links are being collected now.
Found 1002 links for job offers.


In [12]:
links = list(set(links))

print(f'Found {len(links)} unique links for job offers.')

Found 1002 unique links for job offers.


In [13]:
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
post_date = []
num_of_applicants = []
pay_range = []
work_methods = []
work_times = [] 
seniority_level = []
company_size = []
company_industry = []

In [25]:
for i, link in enumerate(links):
    driver.get(link)

    # Initialize placeholders for all items
    job_title = None
    company_name = None
    company_location = None
    post_date_info = None
    num_applicants = None
    pay_range_info = None
    work_method_info = None
    work_time_info = None
    seniority_info = None
    company_size_info = None
    company_industry_info = None

    # Job Title
    try:
        job_title = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//h1[contains(@class, 'job-details-jobs-unified-top-card__job-title')]"))).text
    except TimeoutException:
        pass  # Placeholder value already set
    job_titles.append(job_title)

    # Primary Description Container
    try:
        primary_desc_container = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "job-details-jobs-unified-top-card__primary-description-without-tagline")))
        container_text = primary_desc_container.get_attribute("innerText").split('·')
        
        if container_text[0].strip():
            company_name = container_text[0].strip()
        if len(container_text) > 1:
            company_location = container_text[1].strip()
        if len(container_text) > 2:
            post_date_info = container_text[2].strip()
        if len(container_text) > 3:
            num_applicants = container_text[3].strip()
    except TimeoutException:
        pass  # Placeholder values already set
    
    company_names.append(company_name)
    company_locations.append(company_location)
    post_date.append(post_date_info)
    num_of_applicants.append(num_applicants)

    # Job Insights
    try:
        job_insight_highlighted = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "job-details-jobs-unified-top-card__job-insight--highlight")))
        pay_range_info = job_insight_highlighted.find_element(By.XPATH, ".//span[1]").text
    except (NoSuchElementException, TimeoutException):
        pass  # Placeholder value already set

    try:
        work_method_info = job_insight_highlighted.find_element(By.XPATH, ".//span[contains(@class, 'job-details-jobs-unified-top-card__job-insight-view-model-secondary')][1]").text
    except NoSuchElementException:
        pass  # Placeholder value already set

    try:
        work_time_info = job_insight_highlighted.find_element(By.XPATH, ".//span[contains(@class, 'job-details-jobs-unified-top-card__job-insight-view-model-secondary')][2]").text
    except NoSuchElementException:
        pass  # Placeholder value already set

    try:
        seniority_info = job_insight_highlighted.find_element(By.XPATH, ".//span[contains(@class, 'job-details-jobs-unified-top-card__job-insight-view-model-secondary')][3]").text
    except NoSuchElementException:
        pass  # Placeholder value already set

    pay_range.append(pay_range_info)
    work_methods.append(work_method_info)
    work_times.append(work_time_info)
    seniority_level.append(seniority_info)

    # Company Size and Industry
    try:
        job_insights = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "job-details-jobs-unified-top-card__job-insight")))
        if len(job_insights) > 1:
            split_text = job_insights[1].text.split('·')
            if len(split_text) == 2:
                company_size_info, company_industry_info = split_text
            elif len(split_text) == 1:

                company_size_info = split_text[0].strip()
                company_industry_info = "Industry Not Available"
            else:
                company_size_info = company_industry_info = "Data Not Available"
    except TimeoutException:
        pass  # Placeholder values already set

    company_size.append(company_size_info.strip())
    company_industry.append(company_industry_info.strip())

    print(f'Scraping the Job Offer {i+1} DONE.')
    time.sleep(2)

Scraping the Job Offer 438 DONE.
Scraping the Job Offer 439 DONE.
Scraping the Job Offer 440 DONE.
Scraping the Job Offer 441 DONE.
Scraping the Job Offer 442 DONE.
Scraping the Job Offer 443 DONE.
Scraping the Job Offer 444 DONE.
Scraping the Job Offer 445 DONE.
Scraping the Job Offer 446 DONE.
Scraping the Job Offer 447 DONE.
Scraping the Job Offer 448 DONE.
Scraping the Job Offer 449 DONE.
Scraping the Job Offer 450 DONE.
Scraping the Job Offer 451 DONE.
Scraping the Job Offer 452 DONE.
Scraping the Job Offer 453 DONE.
Scraping the Job Offer 454 DONE.
Scraping the Job Offer 455 DONE.
Scraping the Job Offer 456 DONE.
Scraping the Job Offer 457 DONE.
Scraping the Job Offer 458 DONE.
Scraping the Job Offer 459 DONE.
Scraping the Job Offer 460 DONE.
Scraping the Job Offer 461 DONE.
Scraping the Job Offer 462 DONE.
Scraping the Job Offer 463 DONE.
Scraping the Job Offer 464 DONE.
Scraping the Job Offer 465 DONE.
Scraping the Job Offer 466 DONE.
Scraping the Job Offer 467 DONE.
Scraping t

In [66]:
            
# Creating the dataframe 
df = pd.DataFrame(list(zip(job_titles,company_names,
                    company_locations,post_date, num_of_applicants, pay_range, 
                    work_methods,work_times, seniority_level, company_size, company_industry)),
                    columns =['job_title', 'company_name',
                           'company_location','post_date','applicants','pay', 'work_method','work_time', 'seniority', 'company_size', 'company_industry'])

In [27]:
# Storing the data to csv file
df.to_csv('job_offers.csv', index=False)

In [28]:
driver.quit()

In [67]:
from datetime import datetime, timedelta
import re
import numpy as np

def clean_and_convert_pay(pay_text):
    if pay_text is None:
        return None

    # Normalize the string by removing '$', commas, and converting to lowercase
    normalized_pay_text = pay_text.lower().replace('$', '').replace(',', '')

    # Find all patterns of numbers followed by an optional 'k', '/hr', or '/month'
    # This pattern is looking for a number that may be followed by 'k', '/hr', '/hour', or '/month'
    pay_values = re.findall(r'(\d+(?:\.\d{1,2})?)\s*(k|/hr|/hour|/month)?', normalized_pay_text)

    # Process each found number
    final_values = []
    for value, suffix in pay_values:
        number = float(value)
        # Check for 'k' suffix and multiply by 1,000 for thousands
        if 'k' == suffix:
            number *= 1000
        # Check for '/month' suffix and multiply by 12 for monthly salary
        elif '/month' in suffix:
            number *= 12
        # Check for '/hr' or '/hour' suffix and multiply by 2,080 for hourly wage
        elif '/hr' in suffix or '/hour' in suffix:
            number *= 2080
        final_values.append(number)

    # If any numbers were found, calculate the mean
    if final_values:
        pay_mean = sum(final_values) / len(final_values)
        return pay_mean
    else:
        return None  # Return None for missing or unparseable values



# Function to extract state from location
def extract_state(location_text):
    match = re.search(r'\b([A-Z]{2})\b', location_text)
    return match.group() if match else location_text

# Function to normalize applicants
def normalize_applicants(applicants_text):
    num_applicants = re.search(r'(\d+)', applicants_text.replace(',', ''))
    if num_applicants and int(num_applicants.group(0)) >= 100:
        return '100'
    return num_applicants.group(0) if num_applicants else applicants_text

# Function to count post date as day distance from today
def get_days_from_today(post_date_str):
    if not isinstance(post_date_str, str):
        return None 
    # Current date
    today = datetime.now()

    # Search for numbers and time units in the string
    num_days = re.search(r'\d+', post_date_str)
    time_unit = re.search(r'day|week|month', post_date_str)

    if num_days and time_unit:
        num_days = int(num_days.group())
        time_unit = time_unit.group()

        # Convert the posting date to a timedelta based on the time unit
        if 'day' in time_unit:
            post_date_delta = timedelta(days=num_days)
        elif 'week' in time_unit:
            post_date_delta = timedelta(weeks=num_days)
        elif 'month' in time_unit:
            # Approximate a month as 30 days
            post_date_delta = timedelta(days=num_days*30)
        else:
            # If the time unit is unrecognized, return None
            return None
    else:
        # If the string doesn't contain a recognizable time, return None
        return None

    # Calculate the posting date
    post_date = today - post_date_delta

    # Calculate the difference from today in days
    days_from_today = (today - post_date).days

    return days_from_today



def adjust_location(location):
    # Check the length of the location string
    if len(location) > 2:
        return 'Other'
    else:
        return location


df['work_method'] = df['work_method'].where(df['work_method'].isin(['On-site', 'Hybrid', 'Remote']), np.nan)
df['work_time'] = np.where(df['work_time'] == 'Full-time', df['work_time'], np.nan)
df['pay'] = df['pay'].apply(clean_and_convert_pay)
df['company_location'] = df['company_location'].apply(extract_state)
df['applicants'] = df['applicants'].apply(normalize_applicants)
df['post_date'] = df['post_date'].apply(get_days_from_today)
df['company_location'] = df['company_location'].apply(adjust_location)
# Save the cleaned DataFrame to CSV
df.to_csv('cleaned_job_offers.csv', index=False)

print(df.dtypes)

job_title            object
company_name         object
company_location     object
post_date           float64
applicants           object
pay                 float64
work_method          object
work_time            object
seniority            object
company_size         object
company_industry     object
dtype: object


In [68]:
from pymongo import MongoClient

df = pd.read_csv('cleaned_job_offers.csv')

# Convert the DataFrame to a dictionary
data = df.to_dict("records")

# Set up the MongoDB client
client = MongoClient("mongodb://localhost:27017/")  # Default connection string to local MongoDB instance

# Create a new database or connect to an existing one
db = client["job_offers_database"]

# Create a new collection or connect to an existing one
collection = db["job_offers_collection"]


collection.delete_many({})
# Insert the data into the collection
collection.insert_many(data)


InsertManyResult([ObjectId('6600f3a5bb940919abee5272'), ObjectId('6600f3a5bb940919abee5273'), ObjectId('6600f3a5bb940919abee5274'), ObjectId('6600f3a5bb940919abee5275'), ObjectId('6600f3a5bb940919abee5276'), ObjectId('6600f3a5bb940919abee5277'), ObjectId('6600f3a5bb940919abee5278'), ObjectId('6600f3a5bb940919abee5279'), ObjectId('6600f3a5bb940919abee527a'), ObjectId('6600f3a5bb940919abee527b'), ObjectId('6600f3a5bb940919abee527c'), ObjectId('6600f3a5bb940919abee527d'), ObjectId('6600f3a5bb940919abee527e'), ObjectId('6600f3a5bb940919abee527f'), ObjectId('6600f3a5bb940919abee5280'), ObjectId('6600f3a5bb940919abee5281'), ObjectId('6600f3a5bb940919abee5282'), ObjectId('6600f3a5bb940919abee5283'), ObjectId('6600f3a5bb940919abee5284'), ObjectId('6600f3a5bb940919abee5285'), ObjectId('6600f3a5bb940919abee5286'), ObjectId('6600f3a5bb940919abee5287'), ObjectId('6600f3a5bb940919abee5288'), ObjectId('6600f3a5bb940919abee5289'), ObjectId('6600f3a5bb940919abee528a'), ObjectId('6600f3a5bb940919abee52